# 1. Setup

## 1.1 Install Dependencies

In [2]:
print("hello")

hello


## 1.2 Import Dependencies

In [ ]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt